In [1]:
import os
os.getcwd()
%cd EDTalk/

/home/ldap-users-2/lathifgalih-k/clone-video/edtalk-exp/EDTalk


In [2]:
import sys
print(sys.executable)

/home/is/lathifgalih-k/miniconda3/envs/EDTalk/bin/python


#### Experiment 1 file generation

In [8]:
!python demo_lip_pose.py --source_path res/results_by_facesr/demo_EDTalk_A.png --audio_driving_path test_data/mouth_source.wav --pose_driving_path test_data/pose_source1.mp4 --save_path res/demo_video_cloned.mp4 --face_sr

==> loading model
==> loading data
==> running
[0.27406862 0.45186276 0.27406862]
100%|█████████████████████████████████████████| 251/251 [00:10<00:00, 22.86it/s]
ffmpeg version 4.4.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-2)
  configuration: --prefix=/home/is/lathifgalih-k/miniconda3/envs/EDTalk --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1697113881276/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-avresample --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --en

#### Preprocess Data Video n Generation Video

In [6]:
import os
import subprocess
from pathlib import Path
import shutil
from tqdm import tqdm
import yaml

# Configuration
BASE_DIR = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video"
MUSE_VIDEO_DIR = f"{BASE_DIR}/muse-exp/MuseTalk/data/video"
MUSE_AUDIO_DIR = f"{BASE_DIR}/muse-exp/MuseTalk/data/audio" 
MUSE_CONFIG_DIR = f"{BASE_DIR}/muse-exp/MuseTalk/configs/inference"
EDTALK_DIR = f"{BASE_DIR}/edtalk-exp/EDTalk"
OUTPUT_DIR = f"{EDTALK_DIR}/results"

def extract_first_frame(video_path, output_path):
    """Extract first frame from video as source image"""
    cmd = f'ffmpeg -i "{video_path}" -vf "select=eq(n\\,0)" -q:v 3 "{output_path}" -y'
    result = subprocess.run(cmd, shell=True, capture_output=True)
    return result.returncode == 0

def run_edtalk_inference(source_img, audio_path, output_path):
    """Run EDTalk inference with fixed pose"""
    cmd = f'python demo_lip_pose.py --fix_pose --source_path "{source_img}" --audio_driving_path "{audio_path}" --save_path "{output_path}" --face_sr'
    result = subprocess.run(cmd, shell=True, capture_output=True, cwd=EDTALK_DIR)
    
    # Print error output for debugging
    if result.returncode != 0:
        print(f"Error output: {result.stderr.decode() if result.stderr else 'No error message'}")
        print(f"Stdout: {result.stdout.decode() if result.stdout else 'No stdout'}")
    
    return result.returncode == 0

def load_yaml_config(yaml_path):
    """Load configuration from YAML file"""
    if not os.path.exists(yaml_path):
        print(f"YAML config not found: {yaml_path}")
        return {}
    
    with open(yaml_path, 'r') as file:
        config = yaml.safe_load(file)
    
    return config

def get_audio_type_from_path(audio_path):
    """Extract audio type from path"""
    if "CosyVoice2" in audio_path:
        return "CV"
    elif "XTTSv2" in audio_path:
        return "XT"
    elif "YourTTS" in audio_path:
        return "YT"
    return "Unknown"

def get_dataset_type_from_path(audio_path):
    """Extract dataset type from path"""
    if "VCTK" in audio_path:
        return "VCTK"
    elif "Raw" in audio_path:
        return "Raw"
    return "Unknown"

def get_subject_task_from_audio(audio_path):
    """Extract subject and task from audio filename"""
    filename = os.path.basename(audio_path)
    # Remove prefix (CV_, XT_, YT_) and suffix (_cloned.wav)
    parts = filename.replace("CV_", "").replace("XT_", "").replace("YT_", "").replace("_cloned.wav", "")
    
    if "_VCTK" in parts:
        parts = parts.replace("_VCTK", "")
    
    return parts

def create_output_filename(video_path, audio_path):
    """Create output filename based on video and audio paths"""
    video_name = Path(video_path).stem
    audio_type = get_audio_type_from_path(audio_path)
    subject_task = get_subject_task_from_audio(audio_path)
    
    return f"{video_name}_{audio_type}_{subject_task}_cloned.mp4"

def process_yaml_configs():
    """Process video-audio pairs from YAML configuration files"""
    
    # Create output directories
    os.makedirs(f"{OUTPUT_DIR}/test_raw/v15/RAW", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/test_vctk/v15/VCTK", exist_ok=True)
    
    # Load YAML configurations
    yaml_configs = {
        "test_raw": f"{MUSE_CONFIG_DIR}/test_raw.yaml",
        "test_vctk": f"{MUSE_CONFIG_DIR}/test_vctk.yaml"
    }
    
    for config_name, yaml_path in yaml_configs.items():
        print(f"\nProcessing {config_name} configuration...")
        
        # Load YAML config
        config = load_yaml_config(yaml_path)
        
        if not config:
            print(f"No configuration found for {config_name}")
            continue
        
        # Process each task in the config
        for task_name, task_config in tqdm(config.items(), desc=f"Processing {config_name}"):
            video_path = task_config.get("video_path", "")
            audio_path = task_config.get("audio_path", "")
            
            if not video_path or not audio_path:
                print(f"Invalid config for {task_name}: missing video_path or audio_path")
                continue
            
            # Check if files exist
            if not os.path.exists(video_path):
                print(f"Video not found: {video_path}")
                continue
                
            if not os.path.exists(audio_path):
                print(f"Audio not found: {audio_path}")
                continue
            
            # Determine output path based on dataset type
            dataset_type = get_dataset_type_from_path(audio_path)
            output_filename = create_output_filename(video_path, audio_path)
            
            if dataset_type == "Raw" or config_name == "test_raw":
                output_path = f"{OUTPUT_DIR}/test_raw/v15/RAW/{output_filename}"
            else:  # VCTK
                output_path = f"{OUTPUT_DIR}/test_vctk/v15/VCTK/{output_filename}"
            
            # Skip if output already exists
            if os.path.exists(output_path):
                print(f"Skipping existing: {output_filename}")
                continue
            
            # Extract subject ID from video path
            video_name = Path(video_path).stem
            subject_id = video_name.split('_')[0]  # A002, S001, etc.
            
            # Extract first frame as source image
            temp_source_img = f"{EDTALK_DIR}/temp_{subject_id}_source.jpg"
            if not extract_first_frame(video_path, temp_source_img):
                print(f"Failed to extract frame from {video_path}")
                continue
            
            # Crop source image using EDTalk's crop_image2.py
            cropped_source_img = f"{EDTALK_DIR}/temp_{subject_id}_cropped.jpg"
            try:
                crop_cmd = f'python -c "from data_preprocess.crop_image2 import crop_image; crop_image(\'{temp_source_img}\', \'{cropped_source_img}\')"'
                crop_result = subprocess.run(crop_cmd, shell=True, capture_output=True, cwd=EDTALK_DIR)
                
                if os.path.exists(cropped_source_img):
                    source_img = cropped_source_img
                else:
                    source_img = temp_source_img
                    print(f"Using uncropped source image for {subject_id}")
            except Exception as e:
                source_img = temp_source_img
                print(f"Crop failed for {subject_id}, using original: {e}")
            
            # Run EDTalk inference
            print(f"Processing: {output_filename}")
            success = run_edtalk_inference(source_img, audio_path, output_path)
            
            if success:
                print(f"✓ Generated: {output_path}")
            else:
                print(f"✗ Failed: {output_path}")
            
            # Cleanup temp files
            for temp_file in [temp_source_img, cropped_source_img]:
                if os.path.exists(temp_file):
                    os.remove(temp_file)

def scan_and_process_available_files():
    """Scan available files and process them dynamically"""
    
    # Create output directories
    os.makedirs(f"{OUTPUT_DIR}/test_raw/v15/RAW", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/test_vctk/v15/VCTK", exist_ok=True)
    
    # Scan available video files
    video_folders = {
        "als-input": f"{MUSE_VIDEO_DIR}/als-input",
        "stroke-input": f"{MUSE_VIDEO_DIR}/stroke-input"
    }
    
    # Scan available audio folders
    audio_base_folders = [
        "ALS-Raw", "ALS-VCTK", "Stroke-Raw", "Stroke-VCTK"
    ]
    
    all_tasks = []
    
    # Collect all video-audio pairs
    for video_type, video_folder in video_folders.items():
        if not os.path.exists(video_folder):
            continue
            
        for video_file in os.listdir(video_folder):
            if not video_file.endswith('.mp4'):
                continue
                
            video_path = os.path.join(video_folder, video_file)
            video_name = Path(video_file).stem
            
            # Extract info from video name
            parts = video_name.split('_')
            subject_id = parts[0]  # A002, S001, etc.
            
            # Determine subject type
            if subject_id.startswith('A'):
                subject_type = "ALS"
            elif subject_id.startswith('S'):
                subject_type = "Stroke"
            else:
                continue
            
            # Search for corresponding audio files
            for audio_base_folder in audio_base_folders:
                if not (subject_type in audio_base_folder):
                    continue
                    
                audio_folder_path = f"{MUSE_AUDIO_DIR}/{audio_base_folder}"
                if not os.path.exists(audio_folder_path):
                    continue
                
                # Check each audio type folder
                for audio_type_folder in ["CosyVoice2", "XTTSv2", "YourTTS"]:
                    audio_type_path = f"{audio_folder_path}/{audio_type_folder}"
                    if not os.path.exists(audio_type_path):
                        continue
                    
                    # Find matching audio files
                    for audio_file in os.listdir(audio_type_path):
                        if not audio_file.endswith('.wav'):
                            continue
                        
                        # Check if audio file matches the subject
                        if subject_id in audio_file:
                            audio_path = os.path.join(audio_type_path, audio_file)
                            
                            task = {
                                "video_path": video_path,
                                "audio_path": audio_path,
                                "subject_id": subject_id,
                                "dataset_type": "Raw" if "Raw" in audio_base_folder else "VCTK"
                            }
                            all_tasks.append(task)
    
    print(f"Found {len(all_tasks)} video-audio pairs to process")
    
    # Process all tasks
    for task in tqdm(all_tasks, desc="Processing all tasks"):
        video_path = task["video_path"]
        audio_path = task["audio_path"]
        subject_id = task["subject_id"]
        dataset_type = task["dataset_type"]
        
        # Create output filename
        output_filename = create_output_filename(video_path, audio_path)
        
        if dataset_type == "Raw":
            output_path = f"{OUTPUT_DIR}/test_raw/v15/RAW/{output_filename}"
        else:  # VCTK
            output_path = f"{OUTPUT_DIR}/test_vctk/v15/VCTK/{output_filename}"
        
        # Skip if output already exists
        if os.path.exists(output_path):
            print(f"Skipping existing: {output_filename}")
            continue
        
        # Extract first frame as source image
        temp_source_img = f"{EDTALK_DIR}/temp_{subject_id}_source.jpg"
        if not extract_first_frame(video_path, temp_source_img):
            print(f"Failed to extract frame from {video_path}")
            continue
        
        # Crop source image using EDTalk's crop_image2.py
        cropped_source_img = f"{EDTALK_DIR}/temp_{subject_id}_cropped.jpg"
        try:
            crop_cmd = f'python -c "from data_preprocess.crop_image2 import crop_image; crop_image(\'{temp_source_img}\', \'{cropped_source_img}\')"'
            crop_result = subprocess.run(crop_cmd, shell=True, capture_output=True, cwd=EDTALK_DIR)
            
            if os.path.exists(cropped_source_img):
                source_img = cropped_source_img
            else:
                source_img = temp_source_img
                print(f"Using uncropped source image for {subject_id}")
        except Exception as e:
            source_img = temp_source_img
            print(f"Crop failed for {subject_id}, using original: {e}")
        
        # Run EDTalk inference
        print(f"Processing: {output_filename}")
        success = run_edtalk_inference(source_img, audio_path, output_path)
        
        if success:
            print(f"✓ Generated: {output_path}")
        else:
            print(f"✗ Failed: {output_path}")
        
        # Cleanup temp files
        for temp_file in [temp_source_img, cropped_source_img]:
            if os.path.exists(temp_file):
                os.remove(temp_file)

# Check if YAML configs exist, if not use file scanning approach
yaml_configs_exist = (
    os.path.exists(f"{MUSE_CONFIG_DIR}/test_raw.yaml") and 
    os.path.exists(f"{MUSE_CONFIG_DIR}/test_vctk.yaml")
)

if yaml_configs_exist:
    print("Using YAML configuration files...")
    process_yaml_configs()
else:
    print("YAML configs not found, scanning available files...")
    scan_and_process_available_files()

Using YAML configuration files...

Processing test_raw configuration...


Processing test_raw: 100%|██████████| 60/60 [00:00<00:00, 917.73it/s]


Skipping existing: A002_02_BBP_NORMAL_color_25fps_6sec_CV_A002_BBP_cloned.mp4
Skipping existing: A002_02_BBP_NORMAL_color_25fps_6sec_XT_A002_BBP_cloned.mp4
Skipping existing: A002_02_BBP_NORMAL_color_25fps_6sec_YT_A002_BBP_cloned.mp4
Skipping existing: A006_02_BBP_NORMAL_color_25fps_6sec_CV_A006_BBP_cloned.mp4
Skipping existing: A006_02_BBP_NORMAL_color_25fps_6sec_XT_A006_BBP_cloned.mp4
Skipping existing: A006_02_BBP_NORMAL_color_25fps_6sec_YT_A006_BBP_cloned.mp4
Skipping existing: A008_02_BBP_NORMAL_color_25fps_6sec_CV_A008_BBP_cloned.mp4
Skipping existing: A008_02_BBP_NORMAL_color_25fps_6sec_XT_A008_BBP_cloned.mp4
Skipping existing: A008_02_BBP_NORMAL_color_25fps_6sec_YT_A008_BBP_cloned.mp4
Skipping existing: A010_02_BBP_NORMAL_color_25fps_6sec_CV_A010_BBP_cloned.mp4
Skipping existing: A010_02_BBP_NORMAL_color_25fps_6sec_XT_A010_BBP_cloned.mp4
Skipping existing: A010_02_BBP_NORMAL_color_25fps_6sec_YT_A010_BBP_cloned.mp4
Skipping existing: A011_02_BBP_NORMAL_color_25fps_6sec_CV_A011_B

Processing test_vctk:   0%|          | 0/60 [00:00<?, ?it/s]

Skipping existing: A002_02_BBP_NORMAL_color_25fps_6sec_CV_A002_cloned.mp4
Skipping existing: A002_02_BBP_NORMAL_color_25fps_6sec_XT_A002_cloned.mp4
Skipping existing: A002_02_BBP_NORMAL_color_25fps_6sec_YT_A002_cloned.mp4
Skipping existing: A006_02_BBP_NORMAL_color_25fps_6sec_CV_A006_cloned.mp4
Skipping existing: A006_02_BBP_NORMAL_color_25fps_6sec_XT_A006_cloned.mp4
Skipping existing: A006_02_BBP_NORMAL_color_25fps_6sec_YT_A006_cloned.mp4
Skipping existing: A008_02_BBP_NORMAL_color_25fps_6sec_CV_A008_cloned.mp4
Skipping existing: A008_02_BBP_NORMAL_color_25fps_6sec_XT_A008_cloned.mp4
Skipping existing: A008_02_BBP_NORMAL_color_25fps_6sec_YT_A008_cloned.mp4
Skipping existing: A010_02_BBP_NORMAL_color_25fps_6sec_CV_A010_cloned.mp4
Skipping existing: A010_02_BBP_NORMAL_color_25fps_6sec_XT_A010_cloned.mp4
Skipping existing: A010_02_BBP_NORMAL_color_25fps_6sec_YT_A010_cloned.mp4
Skipping existing: A011_02_BBP_NORMAL_color_25fps_6sec_CV_A011_cloned.mp4
Skipping existing: A011_02_BBP_NORMAL_

Processing test_vctk: 100%|██████████| 60/60 [00:00<00:00, 966.34it/s]

Skipping existing: S005_02_BBP_NORMAL_color_25fps_6sec_CV_S005_cloned.mp4
Skipping existing: S005_02_BBP_NORMAL_color_25fps_6sec_XT_S005_cloned.mp4
Skipping existing: S005_02_BBP_NORMAL_color_25fps_6sec_YT_S005_cloned.mp4
Skipping existing: S006_02_BBP_NORMAL_color_25fps_6sec_CV_S006_cloned.mp4
Skipping existing: S006_02_BBP_NORMAL_color_25fps_6sec_XT_S006_cloned.mp4
Skipping existing: S006_02_BBP_NORMAL_color_25fps_6sec_YT_S006_cloned.mp4
Skipping existing: S007_02_BBP_NORMAL_color_25fps_6sec_CV_S007_cloned.mp4
Skipping existing: S007_02_BBP_NORMAL_color_25fps_6sec_XT_S007_cloned.mp4
Skipping existing: S007_02_BBP_NORMAL_color_25fps_6sec_YT_S007_cloned.mp4
Skipping existing: S008_02_BBP_NORMAL_color_25fps_6sec_CV_S008_cloned.mp4
Skipping existing: S008_02_BBP_NORMAL_color_25fps_6sec_XT_S008_cloned.mp4
Skipping existing: S008_02_BBP_NORMAL_color_25fps_6sec_YT_S008_cloned.mp4
Skipping existing: S009_02_BBP_NORMAL_color_25fps_6sec_CV_S009_cloned.mp4
Skipping existing: S009_02_BBP_NORMAL_

#### Preprocess Data Video n Generation Video with Original Pose

In [3]:
import os
import subprocess
from pathlib import Path
import shutil
from tqdm import tqdm
import yaml

# Configuration
BASE_DIR = "/home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video"
MUSE_VIDEO_DIR = f"{BASE_DIR}/muse-exp/MuseTalk/data/video"
MUSE_AUDIO_DIR = f"{BASE_DIR}/muse-exp/MuseTalk/data/audio" 
MUSE_CONFIG_DIR = f"{BASE_DIR}/muse-exp/MuseTalk/configs/inference"
EDTALK_DIR = f"{BASE_DIR}/edtalk-exp/EDTalk"
OUTPUT_DIR = f"{EDTALK_DIR}/results"

def extract_first_frame(video_path, output_path):
    """Extract first frame from video as source image"""
    cmd = f'ffmpeg -i "{video_path}" -vf "select=eq(n\\,0)" -q:v 3 "{output_path}" -y'
    result = subprocess.run(cmd, shell=True, capture_output=True)
    return result.returncode == 0

def run_edtalk_pose_inference(source_img, audio_path, output_path, source_video):
    """Run EDTalk inference with fixed pose"""
    cmd = f'python demo_lip_pose.py --source_path "{source_img}" --audio_driving_path "{audio_path}" --pose_driving_path "{source_video}" --save_path "{output_path}" --face_sr'
    result = subprocess.run(cmd, shell=True, capture_output=True, cwd=EDTALK_DIR)
    
    # Print error output for debugging
    if result.returncode != 0:
        print(f"Error output: {result.stderr.decode() if result.stderr else 'No error message'}")
        print(f"Stdout: {result.stdout.decode() if result.stdout else 'No stdout'}")
    
    return result.returncode == 0

def load_yaml_config(yaml_path):
    """Load configuration from YAML file"""
    if not os.path.exists(yaml_path):
        print(f"YAML config not found: {yaml_path}")
        return {}
    
    with open(yaml_path, 'r') as file:
        config = yaml.safe_load(file)
    
    return config

def get_audio_type_from_path(audio_path):
    """Extract audio type from path"""
    if "CosyVoice2" in audio_path:
        return "CV"
    elif "XTTSv2" in audio_path:
        return "XT"
    elif "YourTTS" in audio_path:
        return "YT"
    return "Unknown"

def get_dataset_type_from_path(audio_path):
    """Extract dataset type from path"""
    if "VCTK" in audio_path:
        return "VCTK"
    elif "Raw" in audio_path:
        return "Raw"
    return "Unknown"

def get_subject_task_from_audio(audio_path):
    """Extract subject and task from audio filename"""
    filename = os.path.basename(audio_path)
    # Remove prefix (CV_, XT_, YT_) and suffix (_cloned.wav)
    parts = filename.replace("CV_", "").replace("XT_", "").replace("YT_", "").replace("_cloned.wav", "")
    
    if "_VCTK" in parts:
        parts = parts.replace("_VCTK", "")
    
    return parts

def create_output_filename(video_path, audio_path):
    """Create output filename based on video and audio paths"""
    video_name = Path(video_path).stem
    audio_type = get_audio_type_from_path(audio_path)
    subject_task = get_subject_task_from_audio(audio_path)
    
    return f"{video_name}_{audio_type}_{subject_task}_cloned.mp4"

def process_yaml_configs_pose():
    """Process video-audio pairs from YAML configuration files"""
    
    # Create output directories
    os.makedirs(f"{OUTPUT_DIR}/test_raw/v15/RAW", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/test_vctk/v15/VCTK", exist_ok=True)
    
    # Load YAML configurations
    yaml_configs = {
        "test_raw": f"{MUSE_CONFIG_DIR}/test_raw.yaml",
        "test_vctk": f"{MUSE_CONFIG_DIR}/test_vctk.yaml"
    }
    
    for config_name, yaml_path in yaml_configs.items():
        print(f"\nProcessing {config_name} configuration...")
        
        # Load YAML config
        config = load_yaml_config(yaml_path)
        
        if not config:
            print(f"No configuration found for {config_name}")
            continue
        
        # Process each task in the config
        for task_name, task_config in tqdm(config.items(), desc=f"Processing {config_name}"):
            video_path = task_config.get("video_path", "")
            audio_path = task_config.get("audio_path", "")
            
            if not video_path or not audio_path:
                print(f"Invalid config for {task_name}: missing video_path or audio_path")
                continue
            
            # Check if files exist
            if not os.path.exists(video_path):
                print(f"Video not found: {video_path}")
                continue
                
            if not os.path.exists(audio_path):
                print(f"Audio not found: {audio_path}")
                continue
            
            # Determine output path based on dataset type
            dataset_type = get_dataset_type_from_path(audio_path)
            output_filename = create_output_filename(video_path, audio_path)
            
            if dataset_type == "Raw" or config_name == "test_raw":
                output_path = f"{OUTPUT_DIR}/test_raw/pose/RAW/{output_filename}"
            else:  # VCTK
                output_path = f"{OUTPUT_DIR}/test_vctk/pose/VCTK/{output_filename}"
            
            # Skip if output already exists
            if os.path.exists(output_path):
                print(f"Skipping existing: {output_filename}")
                continue
            
            # Extract subject ID from video path
            video_name = Path(video_path).stem
            subject_id = video_name.split('_')[0]  # A002, S001, etc.
            
            # Extract first frame as source image
            temp_source_img = f"{EDTALK_DIR}/temp_{subject_id}_source.jpg"
            if not extract_first_frame(video_path, temp_source_img):
                print(f"Failed to extract frame from {video_path}")
                continue
            
            # Crop source image using EDTalk's crop_image2.py
            cropped_source_img = f"{EDTALK_DIR}/temp_{subject_id}_cropped.jpg"
            try:
                crop_cmd = f'python -c "from data_preprocess.crop_image2 import crop_image; crop_image(\'{temp_source_img}\', \'{cropped_source_img}\')"'
                crop_result = subprocess.run(crop_cmd, shell=True, capture_output=True, cwd=EDTALK_DIR)
                
                if os.path.exists(cropped_source_img):
                    source_img = cropped_source_img
                else:
                    source_img = temp_source_img
                    print(f"Using uncropped source image for {subject_id}")
            except Exception as e:
                source_img = temp_source_img
                print(f"Crop failed for {subject_id}, using original: {e}")
            
            # Run EDTalk inference
            print(f"Processing: {output_filename}")
            success = run_edtalk_pose_inference(source_img, audio_path, output_path, video_path)
            
            if success:
                print(f"✓ Generated: {output_path}")
            else:
                print(f"✗ Failed: {output_path}")
            
            # Cleanup temp files
            for temp_file in [temp_source_img, cropped_source_img]:
                if os.path.exists(temp_file):
                    os.remove(temp_file)

def scan_and_process_available_files_pose():
    """Scan available files and process them dynamically"""
    
    # Create output directories
    os.makedirs(f"{OUTPUT_DIR}/test_raw/pose/RAW", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/test_vctk/pose/VCTK", exist_ok=True)
    
    # Scan available video files
    video_folders = {
        "als-input": f"{MUSE_VIDEO_DIR}/als-input",
        "stroke-input": f"{MUSE_VIDEO_DIR}/stroke-input"
    }
    
    # Scan available audio folders
    audio_base_folders = [
        "ALS-Raw", "ALS-VCTK", "Stroke-Raw", "Stroke-VCTK"
    ]
    
    all_tasks = []
    
    # Collect all video-audio pairs
    for video_type, video_folder in video_folders.items():
        if not os.path.exists(video_folder):
            continue
            
        for video_file in os.listdir(video_folder):
            if not video_file.endswith('.mp4'):
                continue
                
            video_path = os.path.join(video_folder, video_file)
            video_name = Path(video_file).stem
            
            # Extract info from video name
            parts = video_name.split('_')
            subject_id = parts[0]  # A002, S001, etc.
            
            # Determine subject type
            if subject_id.startswith('A'):
                subject_type = "ALS"
            elif subject_id.startswith('S'):
                subject_type = "Stroke"
            else:
                continue
            
            # Search for corresponding audio files
            for audio_base_folder in audio_base_folders:
                if not (subject_type in audio_base_folder):
                    continue
                    
                audio_folder_path = f"{MUSE_AUDIO_DIR}/{audio_base_folder}"
                if not os.path.exists(audio_folder_path):
                    continue
                
                # Check each audio type folder
                for audio_type_folder in ["CosyVoice2", "XTTSv2", "YourTTS"]:
                    audio_type_path = f"{audio_folder_path}/{audio_type_folder}"
                    if not os.path.exists(audio_type_path):
                        continue
                    
                    # Find matching audio files
                    for audio_file in os.listdir(audio_type_path):
                        if not audio_file.endswith('.wav'):
                            continue
                        
                        # Check if audio file matches the subject
                        if subject_id in audio_file:
                            audio_path = os.path.join(audio_type_path, audio_file)
                            
                            task = {
                                "video_path": video_path,
                                "audio_path": audio_path,
                                "subject_id": subject_id,
                                "dataset_type": "Raw" if "Raw" in audio_base_folder else "VCTK"
                            }
                            all_tasks.append(task)
    
    print(f"Found {len(all_tasks)} video-audio pairs to process")
    
    # Process all tasks
    for task in tqdm(all_tasks, desc="Processing all tasks"):
        video_path = task["video_path"]
        audio_path = task["audio_path"]
        subject_id = task["subject_id"]
        dataset_type = task["dataset_type"]
        
        # Create output filename
        output_filename = create_output_filename(video_path, audio_path)
        
        if dataset_type == "Raw":
            output_path = f"{OUTPUT_DIR}/test_raw/pose/RAW/{output_filename}"
        else:  # VCTK
            output_path = f"{OUTPUT_DIR}/test_vctk/pose/VCTK/{output_filename}"
        
        # print(output_path)
        # Skip if output already exists
        if os.path.exists(output_path):
            print(f"Skipping existing: {output_path}")
            continue
        
        # Extract first frame as source image
        temp_source_img = f"{EDTALK_DIR}/temp_{subject_id}_source.jpg"
        if not extract_first_frame(video_path, temp_source_img):
            print(f"Failed to extract frame from {video_path}")
            continue
        
        # Crop source image using EDTalk's crop_image2.py
        cropped_source_img = f"{EDTALK_DIR}/temp_{subject_id}_cropped.jpg"
        try:
            crop_cmd = f'python -c "from data_preprocess.crop_image2 import crop_image; crop_image(\'{temp_source_img}\', \'{cropped_source_img}\')"'
            crop_result = subprocess.run(crop_cmd, shell=True, capture_output=True, cwd=EDTALK_DIR)
            
            if os.path.exists(cropped_source_img):
                source_img = cropped_source_img
            else:
                source_img = temp_source_img
                print(f"Using uncropped source image for {subject_id}")
        except Exception as e:
            source_img = temp_source_img
            print(f"Crop failed for {subject_id}, using original: {e}")
        
        # Run EDTalk inference
        print(f"Processing: {output_filename}")
        success = run_edtalk_pose_inference(source_img, audio_path, output_path, video_path)
        
        if success:
            print(f"✓ Generated: {output_path}")
        else:
            print(f"✗ Failed: {output_path}")
        
        # Cleanup temp files
        for temp_file in [temp_source_img, cropped_source_img]:
            if os.path.exists(temp_file):
                os.remove(temp_file)

# Check if YAML configs exist, if not use file scanning approach
yaml_configs_exist = (
    os.path.exists(f"{MUSE_CONFIG_DIR}/test_raw.yaml") and 
    os.path.exists(f"{MUSE_CONFIG_DIR}/test_vctk.yaml")
)

if yaml_configs_exist:
    print("Using YAML configuration files...")
    process_yaml_configs_pose()
else:
    print("YAML configs not found, scanning available files...")
    scan_and_process_available_files_pose()

Using YAML configuration files...

Processing test_raw configuration...


Processing test_raw:   0%|          | 0/60 [00:00<?, ?it/s]

Processing: A002_02_BBP_NORMAL_color_25fps_6sec_CV_A002_BBP_cloned.mp4


Processing test_raw:   2%|▏         | 1/60 [00:56<55:22, 56.31s/it]

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_raw/pose/RAW/A002_02_BBP_NORMAL_color_25fps_6sec_CV_A002_BBP_cloned.mp4
Processing: A002_02_BBP_NORMAL_color_25fps_6sec_XT_A002_BBP_cloned.mp4


...

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_vctk/pose/VCTK/S013_02_BBP_NORMAL_color_25fps_6sec_CV_S013_cloned.mp4
Processing: S013_02_BBP_NORMAL_color_25fps_6sec_XT_S013_cloned.mp4


Processing test_vctk:  98%|█████████▊| 59/60 [39:40<00:39, 39.03s/it]

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_vctk/pose/VCTK/S013_02_BBP_NORMAL_color_25fps_6sec_XT_S013_cloned.mp4
Processing: S013_02_BBP_NORMAL_color_25fps_6sec_YT_S013_cloned.mp4


Processing test_vctk: 100%|██████████| 60/60 [40:17<00:00, 40.29s/it]

✓ Generated: /home/is/lathifgalih-k/research_naist/multimodal-clone/clone-video/edtalk-exp/EDTalk/results/test_vctk/pose/VCTK/S013_02_BBP_NORMAL_color_25fps_6sec_YT_S013_cloned.mp4
